In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from itertools import cycle, islice
import weightedcalcs as wc

# This line lets us plot on our ipython notebook
%matplotlib inline

In [2]:
#Load data
df_15 = pd.read_table("proj_data/ss15hpa_1yr.csv", sep=',', low_memory=False)

#preview first 5 records...
df_15.head()

,RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,...,wgtp71,wgtp72,wgtp73,wgtp74,wgtp75,wgtp76,wgtp77,wgtp78,wgtp79,wgtp80
0,H,26,2,2600,1,42,1000000,1001264,37,1,...,31,37,33,38,12,11,34,36,33,35
1,H,31,2,3403,1,42,1000000,1001264,206,3,...,205,209,56,216,68,224,235,196,369,319
2,H,35,2,3603,1,42,1000000,1001264,30,2,...,54,51,8,38,31,10,31,9,33,10
3,H,44,2,200,1,42,1000000,1001264,47,2,...,53,46,54,74,54,16,39,16,10,43
4,H,52,2,1200,1,42,1000000,1001264,111,3,...,145,219,24,137,106,164,36,99,119,167


In [3]:
df_15.shape

(63762, 235)

In [4]:
print("total columns:", len(df_15.columns))

total columns: 235


In [5]:
df_15['PUMA'].dtype

dtype('int64')

In [ ]:
### NOTE PULL IN SMOC (selected monthly owner costs) -- Note: Use ADJHSG to adjust SMOCP to constant dollars.
### Pull in NP (Number of person records following this housing record) -- I believe this is right one for determining the poverty guideline...
### Pull in TYPE (type of unit) OR BLD (units in strcuture?)

In [6]:
#too many columns! let's do something about it... 
#get relevant columns: SERIALNO, ST, PUMA, ADJHSG, ADJINC, WGTP, BDSP (# bedrooms), RNTP (rent), TEN (tenure), VACS (vacancy status),
#...HINCP (hh income), GRNTP (gross rent), GRPIP (gross rent as % of hh inc), KIT (complete kitchen), OCPIP (selected monthly owner costs as % hh inc)
#...PLM (complete plumbing) 
df_15_s= df_15[['SERIALNO','ST', 'PUMA', 'ADJHSG', 'WGTP', 'BDSP', 'RNTP', 'TEN', 'VACS','HINCP','GRNTP', 'GRPIP','OCPIP', 'KIT', 'PLM']]

df_15_s.head()

,SERIALNO,ST,PUMA,ADJHSG,WGTP,BDSP,RNTP,TEN,VACS,HINCP,GRNTP,GRPIP,OCPIP,KIT,PLM
0,26,42,2600,1000000,37,2.0,500.0,3.0,NaN,45000.0,603.0,16.0,NaN,1.0,1.0
1,31,42,3403,1000000,206,2.0,800.0,3.0,NaN,43300.0,1147.0,32.0,NaN,1.0,1.0
2,35,42,3603,1000000,30,4.0,NaN,1.0,NaN,125000.0,NaN,NaN,20.0,1.0,1.0
3,44,42,200,1000000,47,3.0,NaN,2.0,NaN,96200.0,NaN,NaN,7.0,1.0,1.0
4,52,42,1200,1000000,111,2.0,1100.0,3.0,NaN,20200.0,1190.0,71.0,NaN,1.0,1.0


In [8]:
# r is for renters...
df_15_r = df_15_s[df_15_s['GRPIP'].notnull()] 
print("PA rental households in data:", len(df_15_r.index))


PA rental households in data: 11170


In [9]:
# renters with complete kitchen and plumbing
df_15_rk = df_15_r[df_15_r['KIT']==1]
df_15_rkp = df_15_rk[df_15_rk['PLM']==1]

print("PA rental households with complete kitchen, plumbing:", len(df_15_rkp.index))

PA rental households with complete kitchen, plumbing: 10954


In [10]:
#add weights to calculator
calc = wc.Calculator("WGTP")

#group by PUMA
grp = df_15_rkp.groupby(["PUMA"])


In [42]:
#calculate bottom 25th quartile of HH income for ea. PUMA
s_inc25 = calc.quantile(grp,"HINCP", 0.25)

#convert panda series to df
dfinc25= s_inc25.to_frame()

#rename column so we know it refers to income threshold for bottom 25%
dfinc25.columns = ["INC25P"]

#merge with our df containing rental households with complete kitchen and plumbing...
df_15_inc = df_15_rkp.merge(dfinc25, left_on='PUMA', right_index=True)

#check that it worked!
df_15_inc.head()


,SERIALNO,ST,PUMA,ADJHSG,WGTP,BDSP,RNTP,TEN,VACS,HINCP,GRNTP,GRPIP,OCPIP,KIT,PLM,INC25P
0,26,42,2600,1000000,37,2.0,500.0,3.0,NaN,45000.0,603.0,16.0,NaN,1.0,1.0,14800.0
421,10197,42,2600,1000000,166,2.0,500.0,3.0,NaN,15600.0,500.0,38.0,NaN,1.0,1.0,14800.0
795,19014,42,2600,1000000,80,3.0,500.0,3.0,NaN,51000.0,822.0,19.0,NaN,1.0,1.0,14800.0
972,23644,42,2600,1000000,59,3.0,800.0,3.0,NaN,77000.0,1030.0,16.0,NaN,1.0,1.0,14800.0
1573,37732,42,2600,1000000,192,3.0,580.0,3.0,NaN,2400.0,832.0,101.0,NaN,1.0,1.0,14800.0


In [44]:
#filter out rows where household income is below bottom 25th quartile threshold
df_15_bottom25= df_15_inc[(df_15_inc['HINCP']<= df_15_inc['INC25P'])]

df_15_bottom25

,SERIALNO,ST,PUMA,ADJHSG,WGTP,BDSP,RNTP,TEN,VACS,HINCP,GRNTP,GRPIP,OCPIP,KIT,PLM,INC25P
1573,37732,42,2600,1000000,192,3.0,580.0,3.0,NaN,2400.0,832.0,101.0,NaN,1.0,1.0,14800.0
6775,161303,42,2600,1000000,50,2.0,380.0,3.0,NaN,9100.0,400.0,53.0,NaN,1.0,1.0,14800.0
7540,180251,42,2600,1000000,78,3.0,700.0,3.0,NaN,12670.0,1243.0,101.0,NaN,1.0,1.0,14800.0
11325,270883,42,2600,1000000,57,2.0,380.0,3.0,NaN,12000.0,1020.0,101.0,NaN,1.0,1.0,14800.0
15488,367919,42,2600,1000000,108,3.0,1400.0,3.0,NaN,12440.0,1770.0,101.0,NaN,1.0,1.0,14800.0
16046,381095,42,2600,1000000,64,2.0,130.0,3.0,NaN,4600.0,160.0,42.0,NaN,1.0,1.0,14800.0
17589,417068,42,2600,1000000,20,1.0,450.0,3.0,NaN,14800.0,500.0,41.0,NaN,1.0,1.0,14800.0
19872,470886,42,2600,1000000,276,3.0,500.0,3.0,NaN,3400.0,783.0,101.0,NaN,1.0,1.0,14800.0
21902,518556,42,2600,1000000,88,3.0,400.0,3.0,NaN,12000.0,836.0,84.0,NaN,1.0,1.0,14800.0
22011,521089,42,2600,1000000,18,4.0,470.0,3.0,NaN,8000.0,680.0,101.0,NaN,1.0,1.0,14800.0
